# VIK DATA CLEANING

- Remember to set Python kernel to 3 (not later).

## VIK EXPLORATION: File operations

In [ ]:
### Copy file's contents to another file; this works for text files...

with open('/Users/vix/Repos/Python-Learning/src/NLP/SNLI Stanford Corpus/README.txt','r') as sourcefile:
    with open('/Users/vix/Repos/Python-Learning/src/NLP/SNLI Stanford Corpus/DESTFILE.txt','w') as destfile:
        for line in sourcefile:
            destfile.write(line)


### And for an image file...simply append `b` for *binary mode* to file operation command `r`, `w`, or `a`

with open('/Users/vix/OneDrive/Temp/Portrait_Vikram_Before-After.png','rb') as sourceimage:
    with open('/Users/vix/Repos/Python-Learning/src/NLP/SNLI Stanford Corpus/destimage.png','wb') as destimage:
        for line in sourceimage:
            destimage.write(line)



## VIK EXPLORATION: Get book text from Project Gutenberg, save to file, and populate list object

In [ ]:
import os, requests, re, urllib

### os.chdir('/Users/vix/Repos/Python-Learning/src/NLP/Texts')
gutenberg_texts = [] # Initialize list

for counter in range(45001,50001): # Loop over each book, which is a reference number

    def get_gutenberg_text():
        url = "https://www.gutenberg.org/files/" + str(counter) + "/" + str(counter) + ".txt"

        try: # Check if URL valid
            webpage = urllib.request.urlopen(url) # Open the webpage containing book text

            # Extract book title and author (author TBD) for file name
            linecount = 1
            for line in webpage:
                m = re.search('Title: ',str(line))
                if m:
                    print("Matched!")
                    text = line.decode()
                    booktitle = text[7 : (len(text) - 2)] # Minus 2 at end critical to remove newline character
                linecount += 1 # Advance line counter
            filename = str(counter)+'_'+booktitle+'.txt'
            
            # Write book text to output file
            print("Currently retrieving: " + booktitle + " -- file name: " + filename)
            with open(str('/Users/vix/Repos/Python-Learning/src/NLP/Corpora/Gutenberg Texts/' + filename),'w') as file:
                webpage = urllib.request.urlopen(url)
                for line in webpage:
                    text = line.decode() # IMP: Extract only text, discarding non-printing characters
                    file.write(text)
            
            # Write book text to list 
            with open(str('/Users/vix/Repos/Python-Learning/src/NLP/Corpora/Gutenberg Texts/' + filename),'r') as file:
                text = [file.read().replace('\n','')]
                gutenberg_texts.append(text)
                print("Added list item: " + str(len(gutenberg_texts)) + "\n") # Enumerate list count, which is number of books
            return gutenberg_texts


        except: # If URL invalid, means no book at that webpage
            print("URL Not Valid\n")

    gutenberg_texts = get_gutenberg_text() # Call function


## VIK EXPLORATION: Separate function to populate list object using existing files in given folder

### Extract book title, author, & text

import os, codecs, re

### Custom function to create `listdir` command that does not show hidden files
def listdir_nohidden(path):
    import glob
    return glob.glob(os.path.join(path, '*'))

os.chdir('/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/books') # Set working folder
print("Adding *all* files in " + os.getcwd() + "\n")
gutenberg_titles = [] # Initialize list of titles
gutenberg_authors = [] # Initialize list of authors
gutenberg_texts = [] # Initialize list of texts

def get_gutenberg_text():
    for file in sorted(listdir_nohidden(".")):
        with codecs.open(file, 'r', encoding='utf-8', errors='ignore') as f:

            # First, take title from file name
            title = re.sub("\.txt","",file)
            title = re.sub("\./","",title)
            title = re.sub("^\d+_","",title)
            gutenberg_titles.append(title) # Put book title in list

            # Then, extract author from book text
            author = ""
            for line in f: 
                if re.search('Author: ',str(line)):
                    author = line[8 : (len(line) - 2)] # Minus 2 at end critical to remove newline character
                    gutenberg_authors.append(author) # Put author in list
                    print("Found author: " + author)
                    break
                elif re.search("^BY ",str(line)):
                    author = line[3 : (len(line) -2)]
                    gutenberg_authors.append(author) # Put author in list
                    print("Found author: " + author)
                    break
            if not author:
                print("No Author Match for " + title)
                gutenberg_authors.append("Unknown Author") # If no author found
                
            # Finally, read all text, beginning with official "START OF" line
            gutenberg_text_current = []
            f.seek(0)
            started = False
            for i, line in enumerate(f.readlines()):
                if not started:
                    if re.search('^\*\*\*.?START OF',str(line)):
                        started = True
                        print ("Started at line", i+1) 
                elif re.search('^\*\*\*.?END OF',str(line)):
                    print ("Ended at line", i+1)
                    break
                else:
                    gutenberg_text_current.append(line)
            
            print(len(gutenberg_text_current))
            if len(gutenberg_text_current) == 0:
                f.seek(0)
                text = f.read()
                gutenberg_text_current.append(text)
                gutenberg_texts.append(gutenberg_text_current)
            else:
                gutenberg_text_current[0 : len(gutenberg_text_current)] = [''.join(gutenberg_text_current[0 : len(gutenberg_text_current)])] 
                gutenberg_texts.append(gutenberg_text_current)

            print("Added list index # " + str(len(gutenberg_texts)-1) + ": " + title + "\n") # Enumerate running count of books

get_gutenberg_text() # Call function

    

### Correct code snippet used to read only specified block of text beginning and ending with some condition

with open("/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/books/11_Alice's Adventures in Wonderland.txt", 'r') as f:

    started = False
    collected_lines = []
    combined_lines = []
    for i, line in enumerate(f.readlines()):
        if not started:
            if re.search('^\*\*\*.?START OF',str(line)):
                started = True
                print ("started at line", i+1) 
        elif re.search('^\*\*\*.?END OF',str(line)):
            print ("end at line", i+1)
            break
        else:
            collected_lines.append(line)
    print("Book done")
    combined_lines[0 : len(collected_lines)] = [''.join(collected_lines[0 : len(collected_lines)])] 

### Prior wrong attempts at above

with codecs.open(file, 'r', encoding='utf-8', errors='ignore') as f:

            f.seek(0)
            started = False
            collected_lines = []
            for i, line in enumerate(f.readlines()):
                    if re.search('^\*\*\*.?START OF',str(line)):
                        started = True
                        print ("started at line", i) 
                        continue
                    if started and re.search('^\*\*\*.?END OF',str(line)):
                        print ("end at line", i)
                        break
                    collected_lines.append(line)

            f.seek(0)
            textstart = 0
            for line in f:
                if re.search('^\*\*\*.?START OF',str(line)):
                    textstart = textstart + len(line) + 4 # Find start position
                    text = f.readline()
                    if re.search('^\*\*\*.?END OF',str(line)):
                        textend = textstart + len(line) + 4 # Find end position
                        break
                else:
                    textstart = textstart + len(line)
                    f.seek(textstart)
            
            text = f.read() # Begin reading at current value of `textstart`
            
            if not text: # In case no starting point found, take it all
                f.seek(0)
                text = f.read()

            text = text.replace('\n',' ')
            text = text.replace('\r','')
            
            gutenberg_texts.append(text)


# Extract book author

import os
import codecs
import re

### Custom function to create `listdir` command that does not show hidden files
def listdir_nohidden(path):
    import glob
    return glob.glob(os.path.join(path, '*'))

os.chdir('/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/books') # Set working folder
print("Adding *all* files in " + os.getcwd() + "\n")
gutenberg_authors = [] # Initialize list of book titles

for file in sorted(listdir_nohidden(".")):
    with codecs.open(file, 'r', encoding='utf-8', errors='ignore') as f:
        author = ""
        linecount = 1
        for line in f:
            if re.search('Author: ',str(line)):
                print("Matched!")
                author = line[8 : (len(line) - 2)] # Minus 2 at end critical to remove newline character
                gutenberg_authors.append(author) # Put authors in list
            linecount += 1 # Advance line counter
        if not author:
            print("No Match for " + file)
            gutenberg_authors.append("Unknown Author") # If no author found


### Useful code to display beginnings of each list item as preview
[book[0][:100] for book in gutenberg_texts]

### Similar code for items in dictionary form
{}

### Useful code to convert MS Word document to text file

import docx2txt
converted_text = docx2txt.process(filename.docx)
with open(Filename.txt, 'w') as file:
    file.write(converted_text)


### Custom function to create `listdir` command that does not show hidden files

def listdir_nohidden(path):
    import glob
    return glob.glob(os.path.join(path, '*'))


### Pickle files for later use - alternative to `csv_writer()`?

### Make a new directory to hold the text files
!mkdir transcripts

for i, c in enumerate(comedians):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file) ### Indexing into the `transcripts` array/list

In [ ]:
### Load pickled files into dictionary data container
#### - ** IMP ** Modified to create dictionary directly from respective list items

books_dict = {} #### `{}` signifies a dictionary
books_dict = dict(zip(gutenberg_titles, gutenberg_texts)) #### `dict()` constructor w/ `zip` iterator


## Useful dictionary syntax

### Dict comprehension creates dictionaries from arbitrary key:value expressions
#### - `test_dict = {x: "This is the text for book " + x for x in gutenberg_titles}`
#### - `test_dict = {key: key*2}
#### - Comprehension also available for object types list and set
### There are three "dictionary views"
**** - `dict.keys()`, `dict.values()`, `dict_items()`

In [ ]:
### Double check to make sure data has been loaded properly
#### `books.keys()` This is the original syntax from the lesson; however, output is a block of running keys.  

### I find the following syntax better, resulting in a more elegant list of keys
list(books_dict)
### or `sorted(books)`
### `'key' in dict` checks for presence of given key in dictionary (or `not in`)

### More checks - need to learn how to display only first few lines of dictionary item
books['Q and the Magic of Grammar'][:1]

## Cleaning The Data

When dealing with numerical data, data cleaning often involves removing null values and duplicate data, dealing with outliers, etc. With text data, there are some common data cleaning techniques, which are also known as text pre-processing techniques.

With text data, this cleaning process can go on forever. There's always an exception to every cleaning step. So, we're going to follow the MVP (minimum viable product) approach - start simple and iterate. Here are a bunch of things you can do to clean your data. We're going to execute just the common cleaning steps here and the rest can be done at a later point to improve our results.

**Common data cleaning steps on all text:**
* Make text all lower case
* Remove punctuation
* Remove numerical values
* Remove common non-sensical text (/n)
* Tokenize text
* Remove stop words

**More data cleaning steps after tokenization:**
* Stemming / lemmatization
* Parts of speech tagging
* Create bi-grams or tri-grams
* Deal with typos
* And more...

### Notice that our dictionary is currently in key: book title, value: list of text format
#### - for some reason our values are not in list of text form - they are already string
### next(iter(books.values()))
books.items()

### - We are going to change this to key: book title, value: string format - NOT USED
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ''.join(list_of_text)
    return combined_text

### Combine it!
books_combined = {key: [combine_text(value)] for (key, value) in books.items()}

In [ ]:
# We can either keep it in dictionary format or put it into a **PANDAS DATAFRAME**
import pandas as pd
import numpy as np
pd.set_option('max_colwidth',150)

books_df = pd.DataFrame.from_dict(books_dict,orient='index')
books_df.columns = ['book_text']
# books_df = books_df.sort_index()
books_df

In [ ]:
### Add author to dataframe as column
books_df['author_name'] = gutenberg_authors
books_df

In [ ]:
### Explore components of dataframe
books_df.columns
# books_df.loc['Q and the Magic of Grammar']


In [ ]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text_round1):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text_round1 = text_round1.lower()
    text_round1 = re.sub('\[.*?\]', '', text_round1)
    text_round1 = re.sub('\*', '', text_round1)
    text_round1 = re.sub('\,', '', text_round1)
    text_round1 = re.sub('\;', '', text_round1)
    text_round1 = re.sub('\:', '', text_round1)
    text_round1 = re.sub('\.', '', text_round1)
    text_round1 = re.sub('\"', '', text_round1)
    text_round1 = re.sub('\w*\d\w*', '', text_round1)
    return text_round1

round1 = lambda x: clean_text_round1(x)# Apply the first round of text cleaning 

In [ ]:
# Let's take a look at the updated text from round 1
books_df_clean = pd.DataFrame(books_df.book_text.apply(round1))
books_df_clean['author_name'] = gutenberg_authors
books_df_clean['book_text']

In [ ]:
# Apply a second round of cleaning
def clean_text_round2(text_round2):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text_round2 = re.sub('[‘’“”…]', '', text_round2)
    text_round2 = re.sub('_', '', text_round2)
    text_round2 = re.sub('\n', ' ', text_round2)
    text_round2 = re.sub('\r', ' ', text_round2)
    return text_round2

round2 = lambda x: clean_text_round2(x) # Apply the second round of text cleaning

In [ ]:
# Let's take a look at the updated text from round 2
books_df_clean = pd.DataFrame(books_df_clean.book_text.apply(round2))
books_df_clean['author_name'] = gutenberg_authors
books_df_clean['book_text']

In [ ]:
# Apply a third round to deal with idiomatic and idiosyncratic deliberate mis-spellings

def clean_text_round3(text_round3):
    text_round3 = re.sub("in\'", 'ing', text_round3)
    text_round3 = re.sub("^\'em", 'them', text_round3)
    text_round3 = re.sub('^.?-', '', text_round3)
    return text_round3

round3 = lambda x: clean_text_round3(x) # Apply the third round of text cleaning

In [ ]:
# Let's take a look at the updated text from round 3
books_df_clean = pd.DataFrame(books_df_clean.book_text.apply(round3))
books_df_clean['author_name'] = gutenberg_authors
books_df_clean

## Organizing The Data

### Corpus

In [ ]:
# Let's take a look at our dataframe
books_df_clean

In [ ]:
# Let's pickle it for later use - original and cleaned
books_df.to_pickle("/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/pickle/books_corpus.pkl")
books_df_clean.to_pickle("/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/pickle/books_clean_corpus.pkl")

### Document-Term Matrix

In [ ]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

## First, make custom stopwords list from previously edited built-in list
books_stopwords = pd.read_csv('/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/books_stopwords.csv', header=None, encoding='utf-8')
books_stopwords = list(np.squeeze(books_stopwords.values))
    # with open("/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/books_stopwords.csv", 'r') as csvfile:
    #     books_stopwords = []
    #     books_stopwords = list(csv.reader(csvfile))

cv = CountVectorizer(stop_words=frozenset(books_stopwords), strip_accents='ascii', token_pattern=r'(?u)\b\w+\b', ngram_range=(1,1))
books_cv = cv.fit_transform(books_df_clean.book_text)
# print(cv.get_feature_names())

## Save resultant words - called feature names - to file
with open("/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/books_featurenames_vocabulary.csv", 'w', newline='') as csvfile:
    vocab = csv.writer(csvfile, delimiter='\n')
    vocab.writerow(cv.get_feature_names())

books_dtm = pd.DataFrame(books_cv.toarray(), columns=cv.get_feature_names())
books_dtm.index = books_df_clean.index
books_dtm


In [ ]:
try:
    print(books_dtm.loc[:, "p"]) ### Check if certain term exists in vocabulary
except:
    print("\nThat text does not exist\n")

In [ ]:
### Let's pickle it for later use
books_dtm.to_pickle("/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/pickle/books_dtm.pkl")

In [ ]:
### Let's also pickle the cleaned data in dataframe form (before we had put it in document-term matrix format), as well as the CountVectorizer object
books_df_clean.to_pickle('/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/pickle/books_df_clean.pkl')
import pickle
pickle.dump(cv, open("/Users/vix/Repos/Python-Learning/src/NLP/Intro to NLP Alice Zhao/pickle/books_cv.pkl", "wb"))